In [0]:
#create a data frame for our original data 
datadf=spark.read.csv("/Volumes/capstone_project/ecommerce/filestore/data.csv",header=True,inferSchema=True)
datadf.show()

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|12/1/2010 8:26|     7.65|     17850|United Kingdom|
|   536365|    21730|GLASS STAR FROSTE...|       6|12/1/2010 8:26|     4.

In [0]:
#create temp view for original data to work with spark sql
datadf.createOrReplaceTempView("tempdf")

In [0]:
#select all the data to store as parquet file and write as parquet file
datafile=spark.sql("select * from tempdf;")
datafile.write.parquet("/Volumes/capstone_project/ecommerce/filestore/parquet_datafile")

In [0]:
#use a data frame to read the parquet file 
parqread=spark.read.parquet("/Volumes/capstone_project/ecommerce/filestore/parquet_datafile")
parqread.show()

+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|    InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|   548311|    22309|TEA COSY RED  STRIPE|       6|3/30/2011 12:05|     2.55|     13728|United Kingdom|
|   548311|    21874|   GIN AND TONIC MUG|      12|3/30/2011 12:05|     1.25|     13728|United Kingdom|
|   548311|    21870|I CAN ONLY PLEASE...|      12|3/30/2011 12:05|     1.25|     13728|United Kingdom|
|   548311|   48173C|DOORMAT BLACK FLOCK |       2|3/30/2011 12:05|     7.95|     13728|United Kingdom|
|   548311|    48187| DOORMAT NEW ENGLAND|       2|3/30/2011 12:05|     7.95|     13728|United Kingdom|
|   548311|    20685|DOORMAT RED RETRO...|       2|3/30/2011 12:05|     7.95|     13728|United Kingdom|
|   548311|    22691|DOORMAT WELCOME S...|       2|3/30/2011 12:

In [0]:
#create tempview for this parquet file this the optimization for queriying on lasrge data sets 
parqread.createOrReplaceTempView("parqdf")

In [0]:
#to find count of customer ids  having the same invoice number
Q7=spark.sql('''select InvoiceNo,COUNT(CustomerID) AS NumCustomers from tempdf group by InvoiceNo   order by InvoiceNo;''')
Q7.show()

+---------+------------+
|InvoiceNo|NumCustomers|
+---------+------------+
|   536365|           7|
|   536366|           2|
|   536367|          12|
|   536368|           4|
|   536369|           1|
|   536370|          20|
|   536371|           1|
|   536372|           2|
|   536373|          16|
|   536374|           1|
|   536375|          16|
|   536376|           2|
|   536377|           2|
|   536378|          19|
|   536380|           1|
|   536381|          35|
|   536382|          12|
|   536384|          13|
|   536385|           7|
|   536386|           3|
+---------+------------+
only showing top 20 rows


In [0]:
#to store data frame in csv format
Q7.coalesce(1).write.csv("/Volumes/capstone_project/ecommerce/filestore/Q7_csv", header=True)

In [0]:
display(
  dbutils.fs.ls("/Volumes/capstone_project/ecommerce/filestore/Q7_csv")
)

path,name,size,modificationTime
dbfs:/Volumes/capstone_project/ecommerce/filestore/Q7_csv/_SUCCESS,_SUCCESS,0,1767521739000
dbfs:/Volumes/capstone_project/ecommerce/filestore/Q7_csv/_committed_6438291678502015095,_committed_6438291678502015095,113,1767521739000
dbfs:/Volumes/capstone_project/ecommerce/filestore/Q7_csv/_started_6438291678502015095,_started_6438291678502015095,0,1767521738000
dbfs:/Volumes/capstone_project/ecommerce/filestore/Q7_csv/part-00000-tid-6438291678502015095-a5b3a07a-2b74-415e-b928-dcb9113c7ed9-261-1-c000.csv,part-00000-tid-6438291678502015095-a5b3a07a-2b74-415e-b928-dcb9113c7ed9-261-1-c000.csv,249465,1767521739000


In [0]:
# to find records having unit price >5
Q16=spark.sql("select distinct * from tempdf where UnitPrice>5;") 
Q16.show() 

+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|    InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|   536749|   90082B|DIAMANTE BOW BROO...|       5|12/2/2010 13:49|     6.35|     17976|United Kingdom|
|   536833|    21621|VINTAGE UNION JAC...|       2|12/2/2010 17:48|      8.5|     18239|United Kingdom|
|   536971|    22844|VINTAGE CREAM DOG...|       4|12/3/2010 13:40|      8.5|     12779|        Poland|
|   537642|    21257|VICTORIAN SEWING ...|       3|12/7/2010 15:33|    16.13|      NULL|United Kingdom|
|   537042|   15056P|EDWARDIAN PARASOL...|       2|12/5/2010 10:45|     5.95|     13838|United Kingdom|
|   537135|    85064|CREAM SWEETHEART ...|       2|12/5/2010 12:35|     5.45|     17059|United Kingdom|
|   537159|    22765|     NEWSPAPER STAND|       1|12/5/2010 13:

In [0]:
Q16.coalesce(1).write.csv("/Volumes/capstone_project/ecommerce/filestore/Q16_csv", header=True)

In [0]:
# customerids having more than 500 invoices each 
Q18=spark.sql("select CustomerID,count(InvoiceNo ) as count from tempdf  group by CustomerID having count(InvoiceNo)>500 order by count(InvoiceNo);")
Q18.show();

+----------+-----+
|CustomerID|count|
+----------+-----+
|     14730|  501|
|     13668|  501|
|     13408|  501|
|     16161|  502|
|     17346|  503|
|     14049|  505|
|     14895|  506|
|     15998|  509|
|     15033|  513|
|     17059|  516|
|     15356|  519|
|     15498|  523|
|     12682|  525|
|     12471|  531|
|     14367|  533|
|     17231|  546|
|     15861|  547|
|     16910|  547|
|     17337|  548|
|     12867|  551|
+----------+-----+
only showing top 20 rows


In [0]:
Q18.coalesce(1).write.csv("/Volumes/capstone_project/ecommerce/filestore/Q18_csv", header=True)

In [0]:
#to find customer ids with stock code 22752
Q19=spark.sql('''select CustomerID from tempdf where StockCode='22752' and CustomerID is not null order by CustomerID;''')
Q19.show()

+----------+
|CustomerID|
+----------+
|     12360|
|     12429|
|     12438|
|     12449|
|     12451|
|     12461|
|     12471|
|     12471|
|     12471|
|     12471|
|     12471|
|     12484|
|     12509|
|     12540|
|     12562|
|     12562|
|     12567|
|     12590|
|     12621|
|     12621|
+----------+
only showing top 20 rows


In [0]:
Q19.coalesce(1).write.csv("/Volumes/capstone_project/ecommerce/filestore/Q19_csv", header=True)

In [0]:
display(
  dbutils.fs.ls("/Volumes/capstone_project/ecommerce/filestore/Q19_csv")
)

path,name,size,modificationTime
dbfs:/Volumes/capstone_project/ecommerce/filestore/Q19_csv/_SUCCESS,_SUCCESS,0,1767522006000
dbfs:/Volumes/capstone_project/ecommerce/filestore/Q19_csv/_committed_6565389370711212207,_committed_6565389370711212207,113,1767522006000
dbfs:/Volumes/capstone_project/ecommerce/filestore/Q19_csv/_started_6565389370711212207,_started_6565389370711212207,0,1767522006000
dbfs:/Volumes/capstone_project/ecommerce/filestore/Q19_csv/part-00000-tid-6565389370711212207-c9736a79-e17d-4e09-9190-5c0e0dbfb02c-326-1-c000.csv,part-00000-tid-6565389370711212207-c9736a79-e17d-4e09-9190-5c0e0dbfb02c-326-1-c000.csv,1847,1767522006000


In [0]:
#stock codes having unit price >10 and count of stock codes 
Q20=spark.sql('''select Stockcode, count(Stockcode) as stockcount, any_value(UnitPrice) as unitprice from tempdf group by Stockcode having any_value(UnitPrice)>10 order by Stockcode;''')
Q20.show()

+---------+----------+---------+
|Stockcode|stockcount|unitprice|
+---------+----------+---------+
|  15056bl|        62|    12.72|
|   15056n|        84|    12.72|
|   15056p|        26|    12.72|
|    20700|        55|    10.17|
|    20704|        45|    14.43|
|    20748|        36|    12.75|
|    20782|        28|    11.02|
|    20785|         6|    39.95|
|    20835|         8|    11.02|
|    20892|         6|    25.49|
|    20934|        37|    11.02|
|    21109|       118|    13.57|
|    21110|       138|    13.57|
|    21120|         2|    16.98|
|    21128|        36|    12.72|
|    21216|       449|    11.02|
|    21217|       352|    21.23|
|    21258|       185|    10.95|
|    21269|        20|    16.98|
|    21277|        24|    21.95|
+---------+----------+---------+
only showing top 20 rows


In [0]:
Q20.coalesce(1).write.csv("/Volumes/capstone_project/ecommerce/filestore/Q20_csv", header=True)